# Cluster analysis tool
Author: Stefan Zeisl, Version: 1.2.0

This tool allows you to select a CSV file from an IVAS cluster analysis export and perform a cluster analysis!

### Step 1: Select file
It is important that each file only contains ONE type of precipiate. Otherwise the calculations will be wrong.

In [3]:
import ipywidgets as widget
from IPython.core.display import display, HTML, clear_output
import numpy as np
from main import ClusterCalcMain
from cluster_analysis_base import ClusterAnalysis
from distance import calculate_mean_distance

style = {'description_width':'auto'}
layout={'width': 'auto'}
uploader = widget.FileUpload(description="Upload a CSV file with cluster analysis data: ",accept='.csv',style=style,layout=layout,multiple=False)
ccm = ClusterCalcMain()
display(uploader)

abc = np.array([r"$\gamma$",r"$\beta$"])

###############Precipiate and Matrix phase UI###############
style = {'description_width':'initial'}

#Filling up
matrix_list = ccm.get_matrix_list()
matrix = widget.Dropdown(options=abc)
phase_list = ccm.get_phase_list()
phase = widget.Dropdown(options=phase_list['Display'])

#Boxes
vb_matrix = widget.VBox([widget.Label("Matrix phase"), matrix])
vb_phase = widget.VBox([widget.Label("Precipitate phase"), phase])
hb_phases = widget.HBox([vb_phase, vb_matrix])
############################################################

###############Additional settings UI#######################
is_bimodal = widget.Checkbox(description="Separate results?",value=False, disabled=False,style=style)
bimodal_value = widget.FloatSlider(description="Diameter separation value",min=0,max=50,disabled=False,style=style,layout={'width':'35em'})
vb_bimodal = widget.VBox([widget.HTML("<b>Separate by diameter</b>"), widget.Label("Allows you to calculate two different statistics separated by precipitate diameter (for bimodal distributions):"), 
                          widget.HBox([is_bimodal, bimodal_value])])
is_distance = widget.Checkbox(description="Calculate inter-precipitate distance?",value=False, disabled=False,style=style)
vb_distance = widget.VBox([widget.HTML("<b>Inter-precipitate distance</b>"),widget.Label("Allows you to calculate the mean distance between the precipitates. This feature is still experimental and may produce incorrect results, especially if clusters are \"touching\" each other."),is_distance])
vb_additional = widget.VBox([vb_bimodal,vb_distance])
############################################################

acc_phases = widget.Accordion(children=[hb_phases],selected_index=None)
acc_additional = widget.Accordion(children=[vb_additional],selected_index=None)
acc_phases.set_title(0,"Precipitate and Matrix phases")
acc_additional.set_title(0,"Additional Settings")
display(widget.VBox([acc_phases,acc_additional]))
start = widget.Button(description="Start",style={"button_color":"orange"})
display(start)

output = widget.Output()

@output.capture()
def on_button_clicked(b):
    #ClusterAnalysis (Data from file, Volume of precipitate phase, Volume of matrix phase)
    clear_output(wait=True)
    file_data = ccm.open_file(uploader)
    ca = ClusterAnalysis(file_data,phase_list.iloc[phase.index]['volume'],matrix_list.iloc[matrix.index]['volume'])
    threshold_value = bimodal_value.value if (is_bimodal.value and bimodal_value.value) else 0
    display(HTML(ca.cluster_calc(threshold_value)))
    if is_distance.value:
        %matplotlib ipympl
        display(HTML(calculate_mean_distance(file_data)))

    
start.on_click(on_button_clicked)
display(output)

FileUpload(value={}, accept='.csv', description='Upload a CSV file with cluster analysis data: ', layout=Layou…

/home/stefan/.local/lib/python3.8/site-packages/traitlets/traitlets.py:586: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  silent = bool(old_value == new_value)


Button(description='Start', style=ButtonStyle(button_color='orange'))

Output()